# Importing Libraries and setting up the client

In [1]:
import pymongo
from pymongo import MongoClient
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords


mongo_client = MongoClient()

client = MongoClient('mongodb://localhost:27017/')
db = client["reddit"]


In [5]:
reddit = praw.Reddit(client_id = "######",
					client_secret = "#######",
					user_agent = "#######",
					username = "########",
					password = "#########")


subreddit = reddit.subreddit('india')

# Scraping Reddit and Saving it to MongoDB Database

In [27]:
# # #List of all the flairs. These will be the keys in classification.
flairs = [ "AskIndia", "Non-Political", "[R]eddiquette",
          "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy",
          "Sports", "Food", "AMA"]


for flair in flairs:

    #The posts' data is collected by searching by the flair name in the list. Top 200 posts are recorded and stored for analysis.

    relevant_subreddits = subreddit.search(flair,limit=200)

    for submission in relevant_subreddits:
        posts = {
        "title":str(submission.title),
        "score":str(submission.score),
        "id":str(submission.id),
        "url":str(submission.url),
        "comms_num":str(submission.num_comments),
        "created":str(submission.created),
        "body":str(submission.selftext),
        "author":str(submission.author),
        "flair":str(flair),
        }

#  Only top ten comments and their authors are considered. 
        submission.comments.replace_more(limit=None)
        comment = ''
        authors = ''
        count = 0
        for top_level_comment in submission.comments:
            comment = comment + ' ' + top_level_comment.body
            authors = authors + ' ' + str(top_level_comment.author)
            count+=1     
            if(count > 10):
                break

        posts["comment"] = str(comment)
        posts["authors"] = str(authors)
        result = db.posts.insert_one(posts)

# Text Cleaning 

In [28]:
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s


# Text Pre-Processing and Saving it into a CSV

In [29]:
collection = db.posts
posts = pd.DataFrame(list(collection.find()))

del posts['_id']

posts['title'] = posts['title'].apply(str)
posts['body'] = posts['body'].apply(str)
posts['comment'] = posts['comment'].apply(str)

posts['title'] = posts['title'].apply(clean_text)
posts['body'] = posts['body'].apply(clean_text)
posts['comment'] = posts['comment'].apply(clean_text)

combined_features = posts["title"] + posts["comment"] + posts["url"] + posts["body"]
posts = posts.assign(combined_features = combined_features)

#saving the csv file
posts.to_csv('data.csv',index=False)